<a href="https://colab.research.google.com/github/ignaciomontovio/TP2Parte2PrograConc/blob/beta/TP_2_Parte2_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programación Concurrente - TP2 - Parte 2

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

---
## 2.1. Ejercicio - Hola Mundo con MPI

### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

### Respuestas

a) La funcion que realiza la instancia maestra (PROCESO MAESTRO) es controlar, coordinar y supervisar el trabajo que llevan a cabo los demas procesos que se implementan (PROCESO ESCLAVO). Ademas se encarga de la asignacion de trabajo, el envio de tareas a los procesos esclavos y de recibir las respuestas que estos le devuelven. Por ultimo se ocupa de marcar la tarjeta y finalizar el trabajo cuando se completan todas las tareas.
Por otro lado la instancia esclava esta a la espera de recibir tareas proporcionadas por el proceso maestro, apenas llegan realiza su trabajo correspondiente y envian los resultados al proceso maestro. Una instancia esclava se da por finalizada cuando recibe el mensaje "END_WORK_FLAG" marcando ademas su tarjeta.

b) main(): Se ejecutara una vez por el proceso maestro mas x cantidad de veces, una por cada proceso esclavo creado.

initWork(): Solo se ejecutara una vez por el proceso maestro.

doWork(): Se ejecutara multiples veces, es decir, una vez por cada proceso esclavo.

c) Los mensajes de trabajo y de finalizacion se diferencian a traves de la utilizacion de etiquetas entre las comunicaciones del proceso mestro y el proceso esclavo. Utilizando WORK_FLAG para enviar tareas de trabajo a los procesos esclavos y END_WORK_FLAG para enviar el mensaje de finalizacion a los procesos esclavos.

d) La funcion BLAS axpy() se utiliza para realizar la siguiente operacion "Y=aX+Y", donde: 
    Y: es un vector (array) al que se le suma una combinación lineal de otro vector X.
    X: es otro vector.
    a: es un escalar que multiplica el vector X antes de sumarse a Y.

La implementacion propuesta seria:

1- El proceso maestro sera el responsable de dividir la operacion axpy() en partes mas pequeñas, es decir, dividir los vectores y distribuir estas partes a los procesos esclavos.

2- Cada proceso esclavo recibira esta parte fragmentada de los datos y realizara la operacion axpy(). Al finalizar enviaran el resultado hacia el proceso maestro.

3- El proceso maestro recibira los reusltados de cada uno de los procesos esclavos y los combinara para obener el resultado final de la operacion axpy()

La eficiencia dependera de la distribucion del trabajo, el tamaño de los vectores y la comunicacion entre los procesos. Lo que si ocurriria seria que dividiriamos el trabajo total por la cantidad de procesos esclavos que tangamos, aprovechando asi el paralelismo.

e) Si se ejecuta una sola instancia no se aprovechara el paralelismo de MPI en la ejecucion. El codigo se ejecutara de manera secuencial en un solo proceso, pudiendose ver el mismo con la instancia maestra.

f) El error del codigo radica en el orden de ejecucion de **comm.recv()** y **stat.Get_tag()**. Debido a que se esta pardiendo el estado final en la ultima operacion de recepcion, ya que no se esta almacenando.

### Codigo modificado

    def doWork(comm):
        while(True):
            stat = MPI.Status() # Obtiene el estado actual del empleado.
            comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
            waitTime =stat.Get_tag()

            if (waitTime == END_WORK_FLAG):
                print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
                return
            else:
                print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

            time.sleep(waitTime)
            comm.send(waitTime, dest=0)

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [1]:
! pip install mpi4py

  Obtaining dependency information for mpi4py from https://files.pythonhosted.org/packages/d6/5e/94811843d8c99729bf2b73341ef74b5874c661b8110dec690fe315dddd7e/mpi4py-3.1.5-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/466.0 kB ? eta -:--:--
    --------------------------------------- 10.2/466.0 kB ? eta -:--:--
    --------------------------------------- 10.2/466.0 kB ? eta -:--:--
   ----- --------------------------------- 71.7/466.0 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 466.0/466.0 kB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\agust\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### 2.1.3. Código del programa en Lenguaje Python

In [2]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   1#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Writing Ejercicio2.py


### 2.1.4 Ejecución del programa

In [3]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   6#@param {type: "number"}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -np $NRO python Ejercicio2.py

'mpirun' is not recognized as an internal or external command,
operable program or batch file.


---
## 3.1 Ejercicio Contar palabras

Desarrollar un programa que permita obtener el valor máximo de un conjunto dado de forma distribuida.

Las condiciones a tener en cuenta son:


*   Debe trabajar con al menos, 4 procesos.
*   El resultado final debe ser informado en cada proceso.
*   Implementar comunicación por Buffer



###3.1.1 Preparación

In [62]:
%%writefile mpi_tp4.py

from mpi4py import MPI
import numpy as np
from IPython.display import display

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

count = 5000
sendbuf = None

if rank == 0:
  sendbuf = np.random.randn(size, count)
  display("Máximo de matriz: ", sendbuf.max())

###### Inicio de código ######
recvbuf = np.empty(count, dtype=np.float64)
comm.Scatter(sendbuf, recvbuf, root=0)

max = recvbuf.max()
recvbuf = np.empty(size, dtype=np.float64)
comm.Gather(max, recvbuf, root=0)

max_max = np.full(1, recvbuf.max())
comm.Bcast(max_max, root=0)

display(rank, " - Máximo de matriz: ", max_max[0])
##############################

Overwriting mpi_tp4.py


In [63]:
# --------------------------------------------
# Formulario
NRO =   10#@param {type: "slider", min: 4, max: 10}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -np $NRO python mpi_tp4.py

Máximo de matriz:  4.291234630478944
0  - Máximo de matriz:  4.291234630478944
1  - Máximo de matriz:  4.291234630478944
2  - Máximo de matriz:  4.291234630478944
6  - Máximo de matriz:  4.291234630478944
4  - Máximo de matriz:  4.291234630478944
8  - Máximo de matriz:  4.291234630478944
3  - Máximo de matriz:  4.291234630478944
7  - Máximo de matriz:  4.291234630478944
5  - Máximo de matriz:  4.291234630478944
9  - Máximo de matriz:  4.291234630478944


---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

